# *k*-Nearest Neighbors

This notebook implements k-nearest neighbors, which is a nonparametric machine learning algorithm. Abbreviated as knn, the algorithm is based on the idea that similar data points are grouped together and close in proximity.

---

First, load the relevant libraries needed.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Import a nice function for plotting decision boundaries
from mlxtend.plotting import plot_decision_regions

# Set the Seaborn theme
sns.set_theme()

### The Data

The model will be trained using the [Hawks](https://r-data.pmagunia.com/dataset/r-dataset-package-stat2data-hawks) dataset. This dataset contains observations for three species of hawks, and attributes such as age, sex, wing length, body weight, tail length, etc. 

The code block below reads the dataset into a pandas DataFrame object, subsets the DataFrame to the relevant variables, and drops any rows where there are missing values for these relevant variables.

In [ ]:
# Read in the data and subset it to the relevant columns/observations
df = pd.read_csv("../../Data/hawks.csv")
df = df[["Species", "Wing", "Tail"]].dropna(axis=0)
df

### Training the model

### Using scikit-learn